In [ ]:
import cv2
from matplotlib import pyplot as plt
from threading import Thread, Condition

In [ ]:
class Camera(Thread):
    def __init__(self):
        Thread.__init__(self)
        self.cond_img_read = Condition()
        self.cap = cv2.VideoCapture(0)
        _, self.buffer = self.cap.read()
        self.print_cam_properties()
        self.set_cam_properties()
        self.print_cam_properties()
        _, self.buffer = self.cap.read()
        self.bgr = None
        self.rgb = None
        self.closed = False
        self.start()
    
    def print_cam_properties(self):
        print("Camera properties:\n" +
            "Buffer: {}\n".format(self.cap.get(cv2.CAP_PROP_BUFFERSIZE)) + # integer
            "Auto Exposure: {}\n".format(self.cap.get(cv2.CAP_PROP_AUTO_EXPOSURE)) + # 0.25 (manual) or 0.75 (auto)
            "Autofocus: {}\n".format(self.cap.get(cv2.CAP_PROP_AUTOFOCUS)) + # 0 or 1?
            "Exposure: {}\n".format(self.cap.get(cv2.CAP_PROP_EXPOSURE)) + # inf
            "Aperture: {}\n".format(self.cap.get(cv2.CAP_PROP_APERTURE)) + # -1
            "Brightness: {}\n".format(self.cap.get(cv2.CAP_PROP_BRIGHTNESS)) + # 0-1
            "Contrast: {}\n".format(self.cap.get(cv2.CAP_PROP_CONTRAST)) + # 0-1
            "Saturation: {}\n".format(self.cap.get(cv2.CAP_PROP_SATURATION)) + # 0-1
            "Hue: {}\n".format(self.cap.get(cv2.CAP_PROP_HUE)) + # 0-1
            "Sharpness: {}\n".format(self.cap.get(cv2.CAP_PROP_SHARPNESS)) + # -1
            "Focus: {}\n".format(self.cap.get(cv2.CAP_PROP_FOCUS)) + # 0-1?
            "FPS: {}\n".format(self.cap.get(cv2.CAP_PROP_FPS)) + # float
            "Gamma: {}\n".format(self.cap.get(cv2.CAP_PROP_GAMMA)) + # -1
            "Gain: {}\n".format(self.cap.get(cv2.CAP_PROP_GAIN)) + # 0-1
            "Height: {}\n".format(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) + # 0-1
            "Width: {}\n".format(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH)) + # 0-1
            "~~~~~~~~~~~~~~~~~~~~~")
    
    def set_cam_properties(self):
        self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        self.cap.set(cv2.CAP_PROP_BRIGHTNESS, 0.5)
        self.cap.set(cv2.CAP_PROP_CONTRAST, 0.5)
        self.cap.set(cv2.CAP_PROP_SATURATION, 0.5)
        self.cap.set(cv2.CAP_PROP_HUE, 0.5)
        self.cap.set(cv2.CAP_PROP_GAIN, 0)
        self.cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.75)
        # self.cap.set(cv2.CAP_PROP_EXPOSURE, 0.5)
    
    def run(self):
        while not self.closed:
            ret, img = self.cap.read()
            # cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)
            with self.cond_img_read:
                self.buffer = img
                self.bgr = None
                self.rgb = None
                self.cond_img_read.notify_all()

    def get_rgb(self):
        if self.rgb is None:
            self.rgb = cv2.cvtColor(self.get_bgr(), cv2.COLOR_BGR2RGB)
        return self.rgb
    
    def get_bgr(self):
        with self.cond_img_read:
            if self.bgr is None:
                if self.buffer is None:
                    self.cond_img_read.wait()
                    assert self.buffer is not None

                self.bgr = self.buffer
            assert self.bgr is not None
            return self.bgr
    
    def wait_for_new(self):
        with self.cond_img_read:
            self.buffer = None
            self.bgr = None
            self.rgb = None

    def close(self):
        self.closed = True
        self.join()
        self.cap.release()

In [ ]:
if __name__ == "__main__":
    cam = Camera()
    for i in range(0, 3):
        rgb = cam.get_rgb()
        plt.imshow(rgb)
        plt.show()
    images = []
    for i in range(0, 10):
        cam.wait_for_new()
        images.append(cam.get_rgb())
    for img in images:
        plt.imshow(img)
        plt.show()
    cam.close()